In [4]:
#Practical 4

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dxc3cv8n
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dxc3cv8n
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=55cd76fcc3b8f81e0bf021369b0c00628ad253535d095655bd443a1dd67ead9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-hkoetem2/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
__global__ void addKernel(int* c, const int* a, const int* b, int size) {
 int i = blockIdx.x * blockDim.x + threadIdx.x;
 if (i < size) {
 c[i] = a[i] + b[i];
 }
}
void addWithCuda(int* c, const int* a, const int* b, int size) {
 int* dev_a = nullptr;
 int* dev_b = nullptr;
 int* dev_c = nullptr;
 cudaMalloc((void**)&dev_c, size * sizeof(int));
 cudaMalloc((void**)&dev_a, size * sizeof(int));
 cudaMalloc((void**)&dev_b, size * sizeof(int));
 cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
 cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);
 addKernel<<<2, (size + 1) / 2>>>(dev_c, dev_a, dev_b, size);

 cudaDeviceSynchronize();
 cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);
 cudaFree(dev_c);
 cudaFree(dev_a);
 cudaFree(dev_b);
}
int main(int argc, char** argv) {
 const int arraySize = 5;
 const int a[arraySize] = { 1, 2, 3, 4, 5 };
 const int b[arraySize] = { 10, 20, 30, 40, 50 };
 int c[arraySize] = { 0 };
 addWithCuda(c, a, b, arraySize);
 printf("{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {%d, %d, %d, %d, %d}\n", c[0], c[1], c[2], c[3], c[4]);
 cudaDeviceReset();
 return 0;
}

{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {11, 22, 33, 44, 55}



In [8]:
%%cu
#include<stdio.h>
#include<cuda.h>
#define row1 2 /* Number of rows of first matrix */
#define col1 3 /* Number of columns of first matrix */
#define row2 3 /* Number of rows of second matrix */
#define col2 2 /* Number of columns of second matrix */
__global__ void matproduct(int *l, int *m, int *n){
 int x = blockIdx.x;
 int y = blockIdx.y;
 int k;
 n[col2 * y + x] = 0;
 for(k = 0; k < col1; k++){
 n[col2 * y + x] = n[col2 * y + x] + l[col1 * y + k] * m[col2 * k + x];
 }
}
int main(){
 //# int row1 = 2, row2 = 3, col1 = 3, col2 = 2;
 int a[row1][col1] = {{1, 2, 3}, {4, 5, 6}};
 int b[row2][col2] = {{9, 8}, {6, 5}, {3, 2}};
 int c[row1][col2];
 int *d, *e, *f;
 int i, j;
 cudaMalloc((void **)&d, row1*col1*sizeof(int));
 cudaMalloc((void **)&e, row2*col2*sizeof(int));
 cudaMalloc((void **)&f, row1*col2*sizeof(int));
 cudaMemcpy(d, a, row1*col1*sizeof(int), cudaMemcpyHostToDevice);
 cudaMemcpy(e, b, row2*col2*sizeof(int), cudaMemcpyHostToDevice);
 dim3 grid(col2, row1);
 # /* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3
grid(no. of columns,no. of rows) */
 matproduct<<<grid,1>>>(d,e,f);
 cudaMemcpy(c, f, row1*col2*sizeof(int), cudaMemcpyDeviceToHost);
 printf("\nProduct of two matrices:\n ");

 for(i = 0; i < row1; i++){
 for(j = 0; j < col2; j++){
 printf("%d\t",c[i][j]);
 }
 printf("\n");
 }
 cudaFree(d);
 cudaFree(e);
 cudaFree(f);
 return 0;
} 



Product of two matrices:
 30	24	
84	69	



In [9]:
#Practical 1

In [18]:
%%writefile main.cpp
#include <bits/stdc++.h>
#include <omp.h>

using namespace std;

const int N = 1e5 + 5;
vector<int> g[N];
bool vis[N];

void bfs(int s) {
    queue<int> q;
    q.push(s);
    vis[s] = true;

    while (!q.empty()) {
        int u = q.front();
        cout << u << " ";
        q.pop();

        #pragma omp parallel for
        for (int i = 0; i < g[u].size(); i++) {
            int v = g[u][i];
            if (!vis[v]) {
                vis[v] = true;
                q.push(v);
            }
        }
    }
}

void dfs(int u) {
    vis[u] = true;
    cout << u << " ";
    

    #pragma omp parallel for
    for (int i = 0; i < g[u].size(); i++) {
        int v = g[u][i];
        if (!vis[v])
            dfs(v);
    }
}

int main() {
    int n, m, s, choice;
    cin >> n >> m >> s >> choice;

    for (int i = 0; i < m; i++) {
        int x, y;
        cin >> x >> y;
        g[x].push_back(y);
        g[y].push_back(x);
    }

    if (choice == 1){
        cout << "BFS : ";
        bfs(s);}
    else if (choice == 2){
        cout << "DFS : ";
        dfs(s);}
    else
        cout << "Invalid choice\n";
    

    return 0;
}


// Test cases:
// Input:
// 6 7 1 1
// 1 2
// 1 3
// 2 4
// 2 5
// 3 5
// 4 6
// 5 6
// Output:
// The result of traversal:
// BFS 1 2 3 4 5 6
// DFS : 1 2 4 6 5 3

Overwriting main.cpp


In [20]:
%%shell
g++ main.cpp -o output
./output

6 7 1 2
1 2
1 3
2 4
2 5
3 5
4 6
5 6
DFS : 1 2 4 6 5 3 

In [2]:
#Practical 2

In [21]:
%%writefile main.cpp
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <omp.h>

using namespace std;

// Parallel Bubble Sort function
void parallelBubbleSort(int *array, int n) {
    int i, j;

    #pragma omp parallel for private(i, j) shared(array)
    for (i = 0; i < n-1; i++) {
        for (j = 0; j < n-i-1; j++) {
            if (array[j] > array[j+1]) {
                // Swap elements
                int temp = array[j];
                array[j] = array[j+1];
                array[j+1] = temp;
            }
        }
    }
}

// Parallel Merge Sort function
void merge(int *array, int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 =  r - m;

    // Create temp arrays
    int *L = new int[n1];
    int *R = new int[n2];

    // Copy data to temp arrays L[] and R[]
    for (i = 0; i < n1; i++)
        L[i] = array[l + i];
    for (j = 0; j < n2; j++)
        R[j] = array[m + 1+ j];

    // Merge the temp arrays back into array[l..r]
    i = 0;
    j = 0;
    k = l;
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            array[k] = L[i];
            i++;
        } else {
            array[k] = R[j];
            j++;
        }
        k++;
    }

    // Copy the remaining elements of L[], if there are any
    while (i < n1) {
        array[k] = L[i];
        i++;
        k++;
    }

    // Copy the remaining elements of R[], if there are any
    while (j < n2) {
        array[k] = R[j];
        j++;
        k++;
    }

    delete [] L;
    delete [] R;
}

void parallelMergeSort(int *array, int l, int r) {
    if (l < r) {
        int m = l+(r-l)/2;

        #pragma omp parallel sections
        {
            #pragma omp section
            parallelMergeSort(array, l, m);

            #pragma omp section
            parallelMergeSort(array, m+1, r);
        }

        merge(array, l, m, r);
    }
}

int main() {
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

int *array = new int[n];
cout << "Enter the elements of the array:" << endl;
for (int i = 0; i < n; i++) {
    cin >> array[i];
}



cout << "Original Array: ";
for (int i = 0; i < n; i++) {
    cout << array[i] << " ";
}
cout << endl;

int choice;
cout << "Enter 1 for Parallel Bubble Sort or 2 for Parallel Merge Sort: ";
cin >> choice;

if (choice == 1) {
    parallelBubbleSort(array, n);
} else if (choice == 2) {
    parallelMergeSort(array, 0, n-1);
} else {
    cout << "Invalid choice. Exiting program." << endl;
    return 0;
}

cout << "Sorted Array: ";
for (int i = 0; i < n; i++) {
    cout << array[i] << " ";
}
cout << endl;

delete [] array;

return 0;
}









// Test Case 1:
// Input:
// Enter the size of the array: 5
// Enter 1 for Parallel Bubble Sort or 2 for Parallel Merge Sort: 1

// Output:
// Original Array: 68 67 69 73 29
// Sorted Array: 29 67 68 69 73

Overwriting main.cpp


In [22]:
%%shell
g++ main.cpp -o output
./output

Enter the size of the array: 5
Enter the elements of the array:
5 1 2 4 7
Original Array: 5 1 2 4 7 
Enter 1 for Parallel Bubble Sort or 2 for Parallel Merge Sort: 1
Sorted Array: 1 2 4 5 7 


In [24]:
%%writefile main.cpp
#include <iostream>
#include <vector>
#include <limits.h>
#include <omp.h>

using namespace std;

void min_reduction(vector<int>& arr) {
    int min_value = INT_MAX;
#pragma omp parallel for reduction(min: min_value)
    for (int i = 0; i < arr.size(); i++) {
        if (arr[i] < min_value) {
            min_value = arr[i];
        }
    }
    cout << "Minimum value: " << min_value << endl;
}

void max_reduction(vector<int>& arr) {
    int max_value = INT_MIN;
#pragma omp parallel for reduction(max: max_value)
    for (int i = 0; i < arr.size(); i++) {
        if (arr[i] > max_value) {
            max_value = arr[i];
        }
    }
    cout << "Maximum value: " << max_value << endl;
}

void sum_reduction(vector<int>& arr) {
    int sum = 0;
#pragma omp parallel for reduction(+: sum)
    for (int i = 0; i < arr.size(); i++) {
        sum += arr[i];
    }
    cout << "Sum: " << sum << endl;
}

void average_reduction(vector<int>& arr) {
    int sum = 0;
#pragma omp parallel for reduction(+: sum)
    for (int i = 0; i < arr.size(); i++) {
        sum += arr[i];
    }
    cout << "Average: " << (double)sum / arr.size() << endl;
}

int main() {
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

    vector<int> arr(n);
    cout << "Enter the elements of the array:" << endl;
    for (int i = 0; i < n; i++) {
        cin >> arr[i];
    }

    min_reduction(arr);
    max_reduction(arr);
    sum_reduction(arr);
    average_reduction(arr);
    return 0;
}

// Output:
// Enter the size of the array: 5
// Enter the elements of the array:
// 5 6 7 2 1
// Minimum value: 1
// Maximum value: 7
// Sum: 21
// Average: 4.2


Overwriting main.cpp


In [25]:
%%shell
g++ main.cpp -o output
./output

Enter the size of the array: 5
Enter the elements of the array:
5 5 5 5 5
Minimum value: 5
Maximum value: 5
Sum: 25
Average: 5
